In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
plt.rcParams['font.sans-serif'] = 'SimHei' # 设置中文显示
plt.rcParams['axes.unicode_minus'] = False

path1 = r"e:\大三\大三上\Optimization in Finance\Final Project\Data\Train Set\Train.xlsx"
path2 = r"e:\大三\大三上\Optimization in Finance\Final Project\Data\Validation Set\Validation.xlsx"
path3 = r"e:\大三\大三上\Optimization in Finance\Final Project\Data\Test Set\Test.xlsx"
df1 = pd.read_excel(path1)
df2 = pd.read_excel(path2)
df3 = pd.read_excel(path3)
stock_codes1 = df1['Symbol'].dropna().astype(str).str.strip().unique().tolist()
stock_codes2 = df2['Symbol'].dropna().astype(str).str.strip().unique().tolist()
stock_codes3 = df3['Symbol'].dropna().astype(str).str.strip().unique().tolist()
common_stock_codes = sorted(set(stock_codes1) & set(stock_codes2) & set(stock_codes3))
print(common_stock_codes)

['1', '100', '156', '157', '166', '1965', '1979', '2', '2001', '2008', '2010', '2027', '2028', '2032', '2044', '2049', '2050', '2051', '2064', '2065', '2074', '2085', '2120', '2129', '2131', '2142', '2152', '2153', '2157', '2174', '2179', '2180', '2183', '2195', '2230', '2236', '2241', '2252', '2269', '2271', '2292', '2294', '2299', '2304', '2310', '2311', '2352', '2371', '2375', '2384', '2400', '2410', '2411', '2414', '2415', '2416', '2422', '2424', '2426', '2429', '2456', '2459', '2460', '2463', '2465', '2466', '2468', '2470', '2475', '2493', '2508', '2555', '2558', '2568', '2572', '2594', '2600', '2601', '2603', '2607', '2608', '2624', '2625', '2648', '2653', '2673', '27', '2709', '2714', '2736', '2739', '2756', '2773', '2791', '2797', '2812', '2821', '2831', '2839', '2841', '2916', '2920', '2925', '2938', '2939', '2945', '2958', '300002', '300003', '300014', '300015', '300017', '300024', '300027', '300033', '300058', '300059', '300070', '300072', '300085', '300122', '300124', '3001

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense
from keras.models import Sequential
def normalize_dataframe(train_df, validation_df):    
    scaler = MinMaxScaler()    
    scaler.fit(train_df)  # 在训练集上拟合归一化模型   
    train_data = pd.DataFrame(scaler.transform(train_df), columns=train_df.columns, index = df_train.index)    
    validation_data = pd.DataFrame(scaler.transform(validation_df), columns=validation_df.columns, index = df_validation.index)    
    return train_data, validation_data,scaler
#2.6滑动窗口定义
def prepare_data(data,win_size,target_feature_idx):
    num_features = data.shape[1]    
    X = []      
    y = []  
    for i in range(len(data)-win_size):
                temp_x = data[i:i + win_size, :] 
                temp_y = data[i + win_size, target_feature_idx]
                X.append(temp_x)
                y.append(temp_y) 
    X = np.asarray(X) 
    y = np.asarray(y)
    return X, y

#目标数据获取
#训练-验证数据
mse=np.zeros(len(common_stock_codes))
rmse=np.zeros(len(common_stock_codes))
r2=np.zeros(len(common_stock_codes))
mae=np.zeros(len(common_stock_codes))
#预测数据
tmse=np.zeros(len(common_stock_codes))
trmse=np.zeros(len(common_stock_codes))
tr2=np.zeros(len(common_stock_codes))
tmae=np.zeros(len(common_stock_codes))



In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
all_results = []
for i in range(0,len(common_stock_codes)):
    file_name=common_stock_codes[i]+'.xlsx'
    df_train = pd.read_excel(f'E:\大三\大三上\Optimization in Finance\Final Project\Data\Train Set\分割\{file_name}', index_col=0, parse_dates=['TradingDate'])
    df_validation = pd.read_excel(f'E:\大三\大三上\Optimization in Finance\Final Project\Data\Validation Set\分割\{file_name}', index_col=0, parse_dates=['TradingDate'])
    df_test = pd.read_excel(f'E:\大三\大三上\Optimization in Finance\Final Project\Data\Test Set\分割\{file_name}', index_col=0, parse_dates=['TradingDate'])
    print(f'已读取{file_name}')

# 调用函数，并接收 scaler
    data_train, data_validation, scaler = normalize_dataframe(
          df_train.loc[:, ['ClosePrice', 'B']], 
         df_validation.loc[:, ['ClosePrice', 'B']]
        )

    win_size = 50 # 时间窗口
    target_feature_idx = 0 # 指定待预测特征列
    train_x, train_y = prepare_data(data_train.values, win_size, target_feature_idx)
    validation_x, validation_y = prepare_data(data_validation.values, win_size, target_feature_idx)
# 归一化测试集（关键！必须在 prepare_data 之前）
    data_test = pd.DataFrame(
    scaler.transform(df_test[['ClosePrice','B']]),
    columns=['ClosePrice','B'],
    index=df_test.index
)
    test_x, test_y = prepare_data(data_test.values, win_size, target_feature_idx)
    print("训练集形状:", train_x.shape, train_y.shape)
    print("测试集形状:", validation_x.shape, validation_y.shape)
    # 此时 df.index 就是纯净的 A 股交易日！
    train_start=df_train.index[50].strftime('%Y-%m-%d')
    train_end=df_train.index[-1].strftime('%Y-%m-%d')
    validation_start=df_validation.index[50].strftime('%Y-%m-%d')
    validation_end=df_validation.index[-1].strftime('%Y-%m-%d')
    test_start=df_test.index[0].strftime('%Y-%m-%d')
    test_end=df_test.index[-1].strftime('%Y-%m-%d')
    train_dates = data_train.loc[train_start:train_end].index
    validation_dates  = data_validation.loc[validation_start:validation_end].index
    test_dates  = data_test.loc[test_start:test_end].index
    # 模型构建
    model = Sequential()
    model.add(LSTM(128, activation='tanh', input_shape=(train_x.shape[1], train_x.shape[2])))
    model.add(Dense(64, activation='tanh'))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(1))
    # 编译模型
    model.compile(loss='mse', optimizer = 'adam')
    # 模型拟合
    history = model.fit(train_x, train_y, epochs=10, batch_size=64, validation_data=(validation_x, validation_y))

    from sklearn import metrics
    y_pred = model.predict(validation_x)
    # 计算均方误差（MSE）
    mse[i] = metrics.mean_squared_error(validation_y, np.array([i for arr in y_pred for i in arr]))
    # 计算均方根误差（RMSE）
    rmse[i] = np.sqrt(mse[i])
    # 计算平均绝对误差（MAE）
    mae[i] = metrics.mean_absolute_error(validation_y, np.array([i for arr in y_pred for i in arr]))
    from sklearn.metrics import r2_score  #拟合优度
    r2[i] = r2_score(validation_y, np.array([i for arr in y_pred for i in arr]))

    #开始预测
    df_hist = pd.concat([df_train, df_validation]).sort_index()
    data_hist = pd.DataFrame(
    scaler.transform(df_hist[['ClosePrice', 'B']]),
    columns=['ClosePrice', 'B'],
    index=df_hist.index
)

    total_steps = len(df_test)
    all_predictions = []

    # 初始序列：使用历史最后 win_size 天
    current_sequence = data_hist.values[-win_size:].copy()  # shape: (win_size, 2)

    for current_start_idx in range(total_steps):
        pred_norm = model.predict(current_sequence.reshape(1, win_size, 2), verbose=0)[0, 0]   
        real_B = df_test['B'].iloc[current_start_idx]
        temp = np.zeros((1, 2))
        temp[0, 0] = pred_norm
        temp[0, 1] = scaler.transform([[0, real_B]])[0, 1] 
        pred_actual = scaler.inverse_transform(temp)[0, 0]
        all_predictions.append(pred_actual)
        real_ClosePrice = df_test['ClosePrice'].iloc[current_start_idx]
        real_step_norm = scaler.transform([[real_ClosePrice, real_B]])[0]  # shape: (2,)
        current_sequence = np.vstack([current_sequence[1:], real_step_norm])

    # 转为 numpy 数组
    all_predictions = np.array(all_predictions)

    # 评估
    actual = df_test['ClosePrice'].values
    tmse[i] = mean_squared_error(actual, all_predictions)
    trmse[i] = np.sqrt(tmse[i])
    tmae[i] = mean_absolute_error(actual, all_predictions)
    tr2[i] = r2_score(actual, all_predictions)

    # 收集结果（不立即写入，避免覆盖）
    stock_result = pd.DataFrame({
        'Symbol': common_stock_codes[i],
        'Actual_ClosePrice': actual,
        'Predicted_ClosePrice': all_predictions
    }, index=df_test.index)
    all_results.append(stock_result)
    print(f"已处理 {common_stock_codes[i]}")



In [ ]:
# ===== 所有股票处理完后，统一写入 Excel =====
final_df = pd.concat(all_results, ignore_index=False)  # 保留日期索引
output_path=r'E:\大三\大三上\Optimization in Finance\Final Project\Result\LSTM_Loop1.xlsx'
final_df.to_excel(output_path, sheet_name='All_Predictions', index=True)
print(f"所有股票预测结果已保存至: {output_path}")

In [6]:
common_stock_codes[243]

'600346'